### Using the built-in chunker

In [4]:
import nltk

text = "Lalbagh Botanical Gardens is a well known botanical garden in Bengaluru, India."
sentences = nltk.sent_tokenize(text)


In [5]:
for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    tags = nltk.pos_tag(words)
    chunks = nltk.ne_chunk(tags)
    print(chunks)

(S
  (PERSON Lalbagh/NNP)
  (PERSON Botanical/NNP Gardens/NNP)
  is/VBZ
  a/DT
  well/RB
  known/VBN
  botanical/JJ
  garden/NN
  in/IN
  (GPE Bengaluru/NNP)
  ,/,
  (GPE India/NNP)
  ./.)


### Writing your own simple chunker

In [6]:
text = "Ravi is the CEO of a Company. He is very powerful public speaker also."

In [11]:
grammar = '\n'.join([
    'NP : {<DT>*<NNP>}',
    'NP : {<JJ>*<NN>}',
    'NP : {<NNP>+}',
])

In [12]:
sentences = nltk.sent_tokenize(text)

In [13]:
for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    tags = nltk.pos_tag(words)
    chunkparser = nltk.RegexpParser(grammar)
    result = chunkparser.parse(tags)
    print(result)

(S
  (NP Ravi/NNP)
  is/VBZ
  (NP the/DT CEO/NNP)
  of/IN
  (NP a/DT Company/NNP)
  ./.)
(S
  He/PRP
  is/VBZ
  very/RB
  (NP powerful/JJ public/JJ speaker/NN)
  also/RB
  ./.)


### Training a chunker

In [14]:
from nltk.corpus import conll2000
from nltk.corpus import treebank_chunk

In [15]:
def mySimpleChunker():
    grammar = 'NP: {<NNP>+}'
    return nltk.RegexpParser(grammar)

In [20]:
def test_nothing(data):
    cp = nltk.RegexpParser("")
    print(cp.evaluate(data))
   

In [17]:
def test_mysimplechunker(data):
    schunker = mySimpleChunker()
    print(schunker.evaluate(data))
    

In [18]:
datasets = [
    conll2000.chunked_sents('test.txt', chunk_types = ['NP']),
    treebank_chunk.chunked_sents()
]


In [21]:
for dataset in datasets:
    test_nothing(dataset[:50])
    test_mysimplechunker(dataset[:50])

ChunkParse score:
    IOB Accuracy:  38.6%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%
ChunkParse score:
    IOB Accuracy:  48.2%%
    Precision:     71.1%%
    Recall:        17.2%%
    F-Measure:     27.7%%
ChunkParse score:
    IOB Accuracy:  45.0%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%
ChunkParse score:
    IOB Accuracy:  50.7%%
    Precision:     51.9%%
    Recall:         8.8%%
    F-Measure:     15.1%%


C:\Users\khana\AppData\Local\Temp\ipykernel_18632\2823061256.py:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(cp.evaluate(data))


### Parsing recursive descent

In [26]:
def RDParserExample(grammar, textlist):
    parser = nltk.parse.RecursiveDescentParser(grammar)
    for text in textlist:
        sentence = nltk.word_tokenize(text)
        for tree in parser.parse(sentence):
            print(tree)
            tree.draw()

In [27]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
NP -> NNP VBZ
VP -> IN NNP | DT NN IN NNP
NNP -> 'Tajmahal' | 'Agra' | 'Bangalore' | 'Karnataka'
VBZ -> 'is'
IN -> 'in' | 'of'
DT -> 'the'
NN -> 'capital'
""")



In [28]:
text = [
    "Tajmahal is in Agra",
    "Bangalore is the capital of Karnataka",
]

In [29]:
RDParserExample(grammar, text)

(S (NP (NNP Tajmahal) (VBZ is)) (VP (IN in) (NNP Agra)))
(S
  (NP (NNP Bangalore) (VBZ is))
  (VP (DT the) (NN capital) (IN of) (NNP Karnataka)))


### Parsing shift-reduce

In [30]:
def SRParserExample(grammar, textlist):
    parser = nltk.parse.ShiftReduceParser(grammar)
    for text in textlist:
        sentence = nltk.word_tokenize(text)
        for tree in parser.parse(sentence):
            print(tree)
            tree.draw()

In [31]:
text = [
    "Tajmahal is in Agra",
    "Bangalore is the capital of Karnataka",
]

In [32]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
NP -> NNP VBZ
VP -> IN NNP | DT NN IN NNP
NNP -> 'Tajmahal' | 'Agra' | 'Bangalore' | 'Karnataka'
VBZ -> 'is'
IN -> 'in' | 'of'
DT -> 'the'
NN -> 'capital'
""")



In [33]:
SRParserExample(grammar , text)

(S (NP (NNP Tajmahal) (VBZ is)) (VP (IN in) (NNP Agra)))


### Parsing dependency grammar and projective dependency

In [34]:
grammar = nltk.grammar.DependencyGrammar.fromstring("""
'savings' -> 'small'
'yield' -> 'savings'
'gains' -> 'large'
'yield' -> 'gains'
""")

In [35]:
sentence = 'small savings yield large gains'

In [36]:
dp  = nltk.parse.ProjectiveDependencyParser(grammar)


In [37]:
for t in sorted(dp.parse(sentence.split())):
    print(t)
    t.draw()

(yield (savings small) (gains large))


### Parsing a chart

In [39]:
from nltk.grammar import CFG
from nltk.parse.chart import ChartParser, BU_LC_STRATEGY

In [40]:
grammar = nltk.CFG.fromstring("""
S -> T1 T4
T1 -> NNP VBZ
T2 -> DT NN
T3 -> IN NNP
T4 -> T3 | T2 T3
NNP -> 'Tajmahal' | 'Agra' | 'Bangalore' | 'Karnataka'
VBZ -> 'is'
IN -> 'in' | 'of'
DT -> 'the'
NN -> 'capital'
""")

In [44]:
cp = ChartParser(grammar, BU_LC_STRATEGY , trace = True )

In [47]:
sentence = "Bangalore is the capital of Karnataka"
tokens = sentence.split()
chart = cp.chart_parse(tokens)
parses = list(chart.parses(grammar.start()))
print("Total Edges :", len(chart.edges()))
for tree in parses:
    print(tree)
    tree.draw()

|.Bangal.  is  . the  .capita.  of  .Karnat.|
|[------]      .      .      .      .      .| [0:1] 'Bangalore'
|.      [------]      .      .      .      .| [1:2] 'is'
|.      .      [------]      .      .      .| [2:3] 'the'
|.      .      .      [------]      .      .| [3:4] 'capital'
|.      .      .      .      [------]      .| [4:5] 'of'
|.      .      .      .      .      [------]| [5:6] 'Karnataka'
|[------]      .      .      .      .      .| [0:1] NNP -> 'Bangalore' *
|[------>      .      .      .      .      .| [0:1] T1 -> NNP * VBZ
|.      [------]      .      .      .      .| [1:2] VBZ -> 'is' *
|[-------------]      .      .      .      .| [0:2] T1 -> NNP VBZ *
|[------------->      .      .      .      .| [0:2] S  -> T1 * T4
|.      .      [------]      .      .      .| [2:3] DT -> 'the' *
|.      .      [------>      .      .      .| [2:3] T2 -> DT * NN
|.      .      .      [------]      .      .| [3:4] NN -> 'capital' *
|.      .      [-------------]      .      .| [2: